In [ ]:
import oracledb
from google.cloud.bigquery import Client
import os, json
from google.cloud import secretmanager
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [ ]:
def set_secrets_as_envs():
  secrets = secretmanager.SecretManagerServiceClient()
  resource_name = f"{os.environ['KNADA_TEAM_SECRET']}/versions/latest"
  secret = secrets.access_secret_version(name=resource_name)
  secret_str = secret.payload.data.decode('UTF-8')
  secrets = json.loads(secret_str)
  os.environ.update(secrets)

In [ ]:
def oracle_secrets():
  set_secrets_as_envs()
  return dict(
    user=os.getenv('DBT_ORCL_USER'),
    password=os.getenv('DBT_ORCL_PASS'),
    host = os.getenv('DBT_ORCL_HOST'),
    service = os.getenv('DBT_ORCL_SERVICE'),
    project_id = os.getenv('BS_PROJECT_ID'), #byttet verdi
    table_uri = os.getenv('TS_v2_TABLE_URI'),  
    encoding="UTF-8",
    nencoding="UTF-8"
  )

oracle_secrets = oracle_secrets()

In [ ]:
def get_data_from_table(max_endret_tid):
    PROJECT_ID = oracle_secrets['project_id'] 
    TABLE_URI = oracle_secrets['table_uri'] 

    client = Client(project=PROJECT_ID)
    job = None
    if max_endret_tid is None:
        job = client.query(f"SELECT * FROM `{TABLE_URI}`")
    else:
        #max_opprettet_dato = pd.to_datetime(max_opprettet_dato, utc=True)
        job = client.query(f"SELECT * FROM `{TABLE_URI}` WHERE endret_tid > '{max_endret_tid}'")
        
    df = job.to_dataframe()
    
    return df

In [ ]:
def convert_to_json(df):
    # Parse the JSON string
    json_str = df.to_json(orient='records')
    #json_objs
    json_objs = json.loads(json_str)

    #list contains tuples of all rows that will be inserted into the table
    extracted_values = []

    for json_obj in json_objs:
        # Extract the value
        json_str = json_obj
        ekstern_behandling_id = json_obj['ekstern_behandling_id']
        opprettet_tid = str(pd.to_datetime(json_obj['opprettet_tid'], unit='ms'))
        endret_tid = str(pd.to_datetime(json_obj['endret_tid'], unit='ms'))
        
        # Append the extracted value to the list
        extracted_values.append((json_str, ekstern_behandling_id, opprettet_tid, endret_tid))

    return extracted_values

In [ ]:
def send_data():
    user = oracle_secrets['user'] + '[dvh_fam_ef]'
    dsn_tns = oracledb.makedsn(oracle_secrets['host'], 1521, service_name = oracle_secrets['service'])
    
    with oracledb.connect(user=user, password = oracle_secrets['password'], dsn=dsn_tns) as conn:
        with conn.cursor() as cursor:
            sql = """select max(endret_tid) from fam_ts_meta_data_v2"""
            cursor.execute(sql)
            result = cursor.fetchall()
            max_endret_tid = result[0][0]

            df = get_data_from_table(max_endret_tid)
            
            extracted_values = convert_to_json(df)
            
            # Insert JSON strings and extracted values into the table
            for json_str, ekstern_behandling_id, opprettet_tid, endret_tid in extracted_values:
                melding = json.dumps(json_str)
                cursor.execute('''
                    INSERT INTO fam_ts_meta_data_v2 (melding, ekstern_behandling_id, opprettet_tid, endret_tid) 
                    VALUES (:melding, :ekstern_behandling_id, to_timestamp(:opprettet_tid , 'YYYY-MM-DD HH24:MI:SS.FF'), to_timestamp(:endret_tid , 'YYYY-MM-DD HH24:MI:SS.FF'))
                ''', (melding, ekstern_behandling_id,opprettet_tid,endret_tid))
            
            # Commit and close connection
            conn.commit()
            #conn.close()

In [ ]:
send_data()